In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import datetime
import time
import numpy as np
from funtions import *
import os
import warnings
from scipy.stats import rankdata
import scipy as sp
from jqdatasdk import *
from numpy import abs
from numpy import log
from numpy import sign
import pymysql
import tushare as ts
warnings.filterwarnings("ignore")
pd.set_option('expand_frame_repr', False)  # 当列太多时不换行
pd.set_option('display.max_rows', 500)  # 最多显示数据的行数

In [2]:
localhost = "192.168.11.250"
username = 'root'
password = "jskjfwq"
dbname = 'futures'
db_port= 3308

In [3]:
now = datetime.datetime.now()
end_date = now.strftime("%Y-%m-%d") #获取到"2021-01-11"这个字符串
end_date = '2022-10-10'

In [4]:
factor_path = r'F:/jswork2/构建聚宽数据库/期货会员持仓/data/汇总/'
index_path = r"F:/jswork2/量化框架/多因子分析框架/data/input_data/"
daily_path = r'F:/jswork2/构建聚宽数据库/期货行情数据/data/日线行情/'
# output_path = r'F:/jswork2/期货策略/期货多因子/非量价因子/会员持仓/分析程序/data/会员因子构建数据/'
# muilt_factor_path = r'F:/jswork2/期货策略/期货多因子/非量价因子/会员持仓/分析程序/data/会员持仓多因子数据/'
# muilt_factor_path = r"F:/jswork2/构建聚宽数据库/期货期限结构构建数据/data/主次力展期数据/"
# muilt_factor_path = r"F:/jswork2/构建聚宽数据库/期货期限结构构建数据/data/主次月展期数据/"
# muilt_factor_path = r"F:/jswork2/构建聚宽数据库/期货期限结构构建数据/data/前5合约展期与回归数据/"
muilt_factor_path = "F:/jswork2/构建聚宽数据库/期货期限结构构建数据/data/近月最大成交量合约的展期数据/"

muilt_factor_path = r"F:/jswork2/构建聚宽数据库/期货基差动量数据/data/近月与最大成交量对的基差动量/"

# muilt_factor_path = r"F:/jswork2/构建聚宽数据库/期货期限结构构建数据/data/带t值得主次月展期数据/"

input_path = r'F:/jswork2/量化框架/选股回测框架/data/input_data/'
output_path = r'F:/jswork2/量化框架/选股回测框架/data/output_data/'
chicang_path = 'F:/jswork2/期货策略/期货多因子/非量价因子/期限结构/分析程序/data/持仓记录/'

In [5]:
# 获取期货交易日历
is_trade_date = pd.read_pickle(index_path + "is_trade_date" + ".pkl")
is_trade_date

,Date
4,2015-01-05
5,2015-01-06
6,2015-01-07
7,2015-01-08
8,2015-01-09
...,...
2826,2022-09-27
2827,2022-09-28
2828,2022-09-29
2829,2022-09-30


In [6]:
index_alldata = is_trade_date.copy()
index_alldata["index_pct_chg"] = 0
index_alldata

,Date,index_pct_chg
4,2015-01-05,0
5,2015-01-06,0
6,2015-01-07,0
7,2015-01-08,0
8,2015-01-09,0
...,...,...
2826,2022-09-27,0
2827,2022-09-28,0
2828,2022-09-29,0
2829,2022-09-30,0


In [7]:
future_list = []
file_list = os.listdir(muilt_factor_path)
for i in file_list:
    future_list.append(i[:-4])
future_list

['A9999.XDCE',
 'AG9999.XSGE',
 'AL9999.XSGE',
 'AP9999.XZCE',
 'AU9999.XSGE',
 'BC9999.XINE',
 'BU9999.XSGE',
 'C9999.XDCE',
 'CF9999.XZCE',
 'CJ9999.XZCE',
 'CU9999.XSGE',
 'CY9999.XZCE',
 'EB9999.XDCE',
 'EG9999.XDCE',
 'FG9999.XZCE',
 'FU9999.XSGE',
 'HC9999.XSGE',
 'I9999.XDCE',
 'J9999.XDCE',
 'JD9999.XDCE',
 'JM9999.XDCE',
 'L9999.XDCE',
 'LH9999.XDCE',
 'LU9999.XINE',
 'M9999.XDCE',
 'MA9999.XZCE',
 'NI9999.XSGE',
 'NR9999.XINE',
 'OI9999.XZCE',
 'P9999.XDCE',
 'PB9999.XSGE',
 'PF9999.XZCE',
 'PG9999.XDCE',
 'PK9999.XZCE',
 'PP9999.XDCE',
 'RB9999.XSGE',
 'RM9999.XZCE',
 'RU9999.XSGE',
 'SA9999.XZCE',
 'SC9999.XINE',
 'SF9999.XZCE',
 'SM9999.XZCE',
 'SN9999.XSGE',
 'SP9999.XSGE',
 'SR9999.XZCE',
 'SS9999.XSGE',
 'TA9999.XZCE',
 'UR9999.XZCE',
 'V9999.XDCE',
 'Y9999.XDCE',
 'ZC9999.XZCE',
 'ZN9999.XSGE']

In [8]:
future_list = [
 'A9999.XDCE',
 'AG9999.XSGE',
 'AL9999.XSGE',
 'AP9999.XZCE',
 'AU9999.XSGE',
 'BU9999.XSGE',
 'C9999.XDCE',
 'CF9999.XZCE',
 'C9999.XDCE',
 'CF9999.XZCE',
 'CJ9999.XZCE',
 'CU9999.XSGE',
 'CY9999.XZCE',
 'EB9999.XDCE',
 'EG9999.XDCE',
 'FG9999.XZCE',
 'FU9999.XSGE',
 'HC9999.XSGE',
 'I9999.XDCE',
 'J9999.XDCE',
 'JD9999.XDCE',
 'JM9999.XDCE',
 'L9999.XDCE',
 'LH9999.XDCE',
 'LU9999.XINE',
 'M9999.XDCE',
 'MA9999.XZCE',
 'NR9999.XINE',
 'OI9999.XZCE',
 'P9999.XDCE',
 'PB9999.XSGE',
 'PF9999.XZCE',
 'PG9999.XDCE',
 'PK9999.XZCE',
 'PP9999.XDCE',
 'RB9999.XSGE',
 'RM9999.XZCE',
 'RU9999.XSGE',
 'SA9999.XZCE',
'SC9999.XINE',
 'SF9999.XZCE',
 'SM9999.XZCE',
 'SP9999.XSGE',
 'SR9999.XZCE',
 'SS9999.XSGE',
 'SN9999.XSGE',
 'TA9999.XZCE',
 'UR9999.XZCE',
 'V9999.XDCE',
 'Y9999.XDCE',
 'ZC9999.XZCE',
 'ZN9999.XSGE',
]

In [9]:
# 获取因子数据
# 交易所不公布原油的持仓
# start_date = "2015-01-01"
# all_factor_data = pd.DataFrame()
# for code in future_list:
# #     code = code.replace("9999", "8888")
#     temp_df = pd.read_pickle(muilt_factor_path+code + ".pkl")
#     temp_df['code'] = temp_df['code'].apply(lambda x: x[:-4])
    
#     all_factor_data = all_factor_data.append(temp_df)

# all_factor_data.rename(columns={"day":"Date", "code":"Ticker"},inplace=True)
# del all_factor_data['zhulicode']
# del all_factor_data['cizhulicode']
# del all_factor_data['jin_price']
# del all_factor_data['ci_jin_price']
# del all_factor_data['diff_days']
# all_factor_data = all_factor_data[all_factor_data['Date']>=start_date]
# all_factor_data.reset_index(drop=True,inplace=True)
# all_factor_data

In [10]:
# 获取近远月因子数据
# 交易所不公布原油的持仓
# start_date = "2015-01-01"
# all_factor_data = pd.DataFrame()
# for code in future_list:
# #     code = code.replace("9999", "8888")
#     temp_df = pd.read_pickle(muilt_factor_path+code + ".pkl")
#     temp_df['code'] = temp_df['code'].apply(lambda x: x[:-4])
    
#     all_factor_data = all_factor_data.append(temp_df)

# all_factor_data.rename(columns={"day":"Date", "code":"Ticker"},inplace=True)
# del all_factor_data['jinyuecode']
# del all_factor_data['cijinyuecode']
# del all_factor_data['jin_price']
# del all_factor_data['ci_jin_price']
# del all_factor_data['diff_days']
# all_factor_data = all_factor_data[all_factor_data['Date']>=start_date]
# all_factor_data.reset_index(drop=True,inplace=True)
# all_factor_data

In [11]:
# 获取回归因子数据
# start_date = "2015-01-01"
# all_factor_data = pd.DataFrame()
# for code in future_list:
# #     code = code.replace("9999", "8888")
#     temp_df = pd.read_pickle(muilt_factor_path+code + ".pkl")
    
#     for n in [5,10,15,20,25,30,35,50]:
#         temp_df[f"ma{n}_rollover_benefits"] = temp_df['rollover_benefits'].rolling(n,min_periods=5).mean()
    
#     all_factor_data = all_factor_data.append(temp_df)

# del all_factor_data['vol_code']
# del all_factor_data['vol_close']
# del all_factor_data['date_code']
# del all_factor_data['date_close']

# all_factor_data = all_factor_data[all_factor_data['Date']>=start_date]
# all_factor_data.reset_index(drop=True,inplace=True)
# all_factor_data

In [12]:
# 获取近月 与最大成立量的合约对展期数据
# start_date = "2015-01-01"
# all_factor_data = pd.DataFrame()
# for code in future_list:
# #     code = code.replace("9999", "8888")
#     temp_df = pd.read_pickle(muilt_factor_path+code + ".pkl")

#     all_factor_data = all_factor_data.append(temp_df)

# del all_factor_data['jin_yue_code']
# del all_factor_data['cijin_yue_code']
# del all_factor_data['jin_yue_close']
# del all_factor_data['cijin_yue_close']
# del all_factor_data['diff_days']

# del all_factor_data['vol_code']
# del all_factor_data['vol_close']
# del all_factor_data['vol_vol']
# del all_factor_data['date_code']
# del all_factor_data['date_close']
# del all_factor_data['date_vol']

# all_factor_data = all_factor_data[all_factor_data['Date']>=start_date]
# all_factor_data.reset_index(drop=True,inplace=True)
# all_factor_data

In [13]:
# 获取近月与最大成立量的合约对基差动量数据
start_date = "2015-01-01"
all_factor_data = pd.DataFrame()
for code in future_list:
#     code = code.replace("9999", "8888")
    temp_df = pd.read_pickle(muilt_factor_path+code + ".pkl")
    all_factor_data = all_factor_data.append(temp_df)
    
del all_factor_data['vol_code']
del all_factor_data['vol_close']
del all_factor_data['vol_vol']

del all_factor_data['date_code']
del all_factor_data['date_close']
del all_factor_data['date_vol']

del all_factor_data['jin_yue_code']
del all_factor_data['cijin_yue_code']
del all_factor_data['jin_yue_close']
del all_factor_data['cijin_yue_close']


for N in [3, 5, 10, 15, 20, 25, 30, 50, 80, 100, 120, 150]:
    del all_factor_data[f'vol_pct_chg{N}']
    del all_factor_data[f'date_pct_chg{N}']
    del all_factor_data[f'jin_yue_pct_chg{N}']
    del all_factor_data[f'cijin_yue_pct_chg{N}']

all_factor_data = all_factor_data[all_factor_data['Date']>=start_date]
all_factor_data.reset_index(drop=True,inplace=True)
all_factor_data

,Date,Ticker,3_basis_momentum,5_basis_momentum,10_basis_momentum,15_basis_momentum,20_basis_momentum,25_basis_momentum,30_basis_momentum,50_basis_momentum,80_basis_momentum,100_basis_momentum,120_basis_momentum,150_basis_momentum
0,2015-01-05,A9999.XDCE,1.306467e-02,0.000284,-0.028608,-0.017011,-0.025508,-0.011772,-0.008149,0.017587,0.050647,0.056404,0.059717,0.067155
1,2015-01-06,A9999.XDCE,1.295603e-02,0.002114,-0.018588,-0.016105,-0.012149,-0.007216,-0.002859,0.017658,0.055616,0.061588,0.060184,0.068445
2,2015-01-07,A9999.XDCE,4.748904e-03,0.010654,-0.020318,-0.022891,-0.017504,-0.020219,-0.004328,0.010618,0.049085,0.053990,0.055300,0.064527
3,2015-01-08,A9999.XDCE,5.062366e-03,0.015316,-0.011247,-0.020676,-0.015293,-0.014160,-0.007133,0.024076,0.059060,0.061989,0.062466,0.074866
4,2015-01-09,A9999.XDCE,-1.120086e-03,0.008787,-0.001431,-0.026470,-0.015660,-0.026935,-0.018675,0.021654,0.053825,0.058060,0.059666,0.070893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81851,2022-11-17,ZN9999.XSGE,3.886065e-04,-0.002573,-0.010982,0.003853,-0.010856,-0.003926,-0.001786,0.004742,0.004539,0.010743,0.011805,0.009593
81852,2022-11-18,ZN9999.XSGE,1.072397e-03,-0.002046,-0.010064,0.002101,-0.003001,-0.002076,0.003331,0.003943,0.006812,0.012342,0.010058,0.006838
81853,2022-11-21,ZN9999.XSGE,4.859077e-03,0.003256,-0.007974,-0.000788,0.005435,-0.001844,-0.001194,0.006041,0.012098,0.014751,0.020089,0.009567
81854,2022-11-22,ZN9999.XSGE,5.950805e-07,0.000223,-0.007732,-0.008154,0.005190,-0.009109,-0.008727,0.002074,0.009497,0.022383,0.013980,0.009242


In [14]:
# 获取期货的指数日线行情数据
rename_dict = {
    'trade_date':'Date',
    'security':'Ticker',
    'open':'Open',
    'close':'Close',
    'high':'High',
    'low':'Low',
    'volume':'Volume',
    'money':'Value' 
}
all_trade_df = pd.read_pickle(input_path+"post_zhulian_data.pkl")
all_trade_df = all_trade_df[all_trade_df['paused']==0]
all_trade_df.rename(columns=rename_dict,inplace=True)
all_trade_df = all_trade_df[all_trade_df["Ticker"].isin(future_list)]
all_trade_df['Date'] = all_trade_df['Date'].astype(str)
all_trade_df = all_trade_df[['Date', 'Ticker','Open','Close','Low','High','Volume','Value','high_limit']]
all_trade_df

,Date,Ticker,Open,Close,Low,High,Volume,Value,high_limit
0,2015-01-05,A9999.XDCE,3938.36,3946.25,3901.50,3952.40,72400.0,3.248361e+09,4111.23
1,2015-01-06,A9999.XDCE,3948.89,3951.52,3936.60,3991.01,176706.0,7.977943e+09,4093.68
2,2015-01-07,A9999.XDCE,3961.17,3953.27,3931.33,3973.46,102236.0,4.604084e+09,4119.13
3,2015-01-08,A9999.XDCE,3957.66,3951.52,3940.11,3969.95,93610.0,4.220491e+09,4109.47
4,2015-01-09,A9999.XDCE,3955.91,3956.78,3936.60,3965.56,106146.0,4.779245e+09,4113.86
...,...,...,...,...,...,...,...,...,...
1883,2022-09-27,ZN9999.XSGE,24413.30,24577.40,24341.50,24808.10,162221.0,1.943660e+10,27632.90
1884,2022-09-28,ZN9999.XSGE,24654.30,23849.40,23849.40,24654.30,236290.0,2.789449e+10,27514.90
1885,2022-09-29,ZN9999.XSGE,24013.40,24387.70,24013.40,24459.40,182021.0,2.153816e+10,27109.90
1886,2022-09-30,ZN9999.XSGE,24690.10,24269.80,24259.50,24792.70,172268.0,2.057620e+10,27171.50


In [15]:
final_data = get_backtest_data(all_trade_df, all_factor_data, is_trade_date)
final_data

A9999.XDCE
AG9999.XSGE
AL9999.XSGE
AP9999.XZCE
AU9999.XSGE
BU9999.XSGE
C9999.XDCE
CF9999.XZCE
CJ9999.XZCE
CU9999.XSGE
EB9999.XDCE
EG9999.XDCE
FG9999.XZCE
FU9999.XSGE
HC9999.XSGE
I9999.XDCE
J9999.XDCE
JD9999.XDCE
JM9999.XDCE
L9999.XDCE
LH9999.XDCE
LU9999.XINE
M9999.XDCE
MA9999.XZCE
NR9999.XINE
OI9999.XZCE
P9999.XDCE
PB9999.XSGE
PF9999.XZCE
PG9999.XDCE
PK9999.XZCE
PP9999.XDCE
RB9999.XSGE
RM9999.XZCE
RU9999.XSGE
SA9999.XZCE
SC9999.XINE
SF9999.XZCE
SM9999.XZCE
SN9999.XSGE
SP9999.XSGE
SR9999.XZCE
SS9999.XSGE
TA9999.XZCE
UR9999.XZCE
V9999.XDCE
Y9999.XDCE
ZC9999.XZCE
ZN9999.XSGE


,Date,Ticker,Open,Close,Low,High,Volume,Value,high_limit,pct_chg,open_chg,3_basis_momentum,5_basis_momentum,10_basis_momentum,15_basis_momentum,20_basis_momentum,25_basis_momentum,30_basis_momentum,50_basis_momentum,80_basis_momentum,100_basis_momentum,120_basis_momentum,150_basis_momentum,is_trade,open_lim_up,next_pct_chg,future_Ndays_chg,tom_istrade,next_day_open_lim_up,tom_open_chg
1,2015-01-06,A9999.XDCE,3948.89,3951.52,3936.60,3991.01,176706.0,7.977943e+09,4093.68,0.001335,[0.0006660099420343624],0.012956,0.002114,-0.018588,-0.016105,-0.012149,-0.007216,-0.002859,0.017658,0.055616,0.061588,0.060184,0.068445,1,False,-0.010424,"[-0.00199436025214772, -0.010424200828867858, ...",1.0,False,[-0.00199436025214772]
2,2015-01-07,A9999.XDCE,3961.17,3953.27,3931.33,3973.46,102236.0,4.604084e+09,4119.13,0.000443,[-0.00199436025214772],0.004749,0.010654,-0.020318,-0.022891,-0.017504,-0.020219,-0.004328,0.010618,0.049085,0.053990,0.055300,0.064527,1,False,0.001873,"[-0.0015514217997503188, -0.010424200828867858...",1.0,False,[-0.0015514217997503188]
3,2015-01-08,A9999.XDCE,3957.66,3951.52,3940.11,3969.95,93610.0,4.220491e+09,4109.47,-0.000443,[-0.0015514217997503188],0.005062,0.015316,-0.011247,-0.020676,-0.015293,-0.014160,-0.007133,0.024076,0.059060,0.061989,0.062466,0.074866,1,False,0.000000,"[0.00021992411354165853, -0.010424200828867858...",1.0,False,[0.00021992411354165853]
4,2015-01-09,A9999.XDCE,3955.91,3956.78,3936.60,3965.56,106146.0,4.779245e+09,4113.86,0.001331,[0.00021992411354165853],-0.001120,0.008787,-0.001431,-0.026470,-0.015660,-0.026935,-0.018675,0.021654,0.053825,0.058060,0.059666,0.070893,1,False,-0.023598,"[-0.00022230418285529296, -0.01042420082886785...",1.0,False,[-0.00022230418285529296]
5,2015-01-12,A9999.XDCE,3958.54,3957.66,3946.25,3982.23,134590.0,6.084913e+09,4108.60,0.000222,[-0.00022230418285529296],0.003797,0.001374,0.001679,-0.027297,-0.015670,-0.024185,-0.010388,0.015367,0.053901,0.056841,0.059554,0.070992,1,False,-0.001674,"[-0.003997711253342029, -0.010424200828867858,...",1.0,False,[-0.003997711253342029]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,2022-09-27,ZN9999.XSGE,24413.30,24577.40,24341.50,24808.10,162221.0,1.943660e+10,27632.90,0.006087,[0.006721745933569112],0.001315,0.000701,-0.000101,0.003647,0.004376,0.004698,0.001355,0.006309,0.009830,0.015849,0.014354,0.015923,1,False,NaN,"[-0.03264744892371707, 0.02257079842679488, -0...",1.0,False,[-0.03264744892371707]
1884,2022-09-28,ZN9999.XSGE,24654.30,23849.40,23849.40,24654.30,236290.0,2.789449e+10,27514.90,-0.029621,[-0.03264744892371707],0.003591,0.007104,0.003787,0.007001,0.007908,0.008320,0.007503,0.009275,0.017489,0.017881,0.021758,0.006196,1,False,NaN,"[0.015587130518793568, -0.004834404228361122, ...",1.0,False,[0.015587130518793568]
1885,2022-09-29,ZN9999.XSGE,24013.40,24387.70,24013.40,24459.40,182021.0,2.153816e+10,27109.90,0.022571,[0.015587130518793568],0.001473,0.004254,0.003571,0.005454,0.008235,0.007108,0.007667,0.009608,0.016469,0.018652,0.026621,0.019410,1,False,NaN,"[-0.017023017322732592, 0.026823459608237288, ...",1.0,False,[-0.017023017322732592]
1886,2022-09-30,ZN9999.XSGE,24690.10,24269.80,24259.50,24792.70,172268.0,2.057620e+10,27171.50,-0.004834,[-0.017023017322732592],0.012926,0.012587,0.010224,0.015396,0.017874,0.016675,0.017468,0.019563,0.027012,0.033058,0.018370,0.035908,1,False,NaN,"[0.02466181489247976, nan, nan, nan, nan, nan,...",1.0,False,[0.02466181489247976]


In [16]:
final_data.to_pickle(output_path + 'jicha_jin_vol_sel_sym_all_post_final_data.pkl')

In [17]:
# final_data = pd.read_pickle(output_path + 'rolling_all_post_final_data.pkl')
# final_data = pd.read_pickle(output_path + 'OLS_rolling_all_post_final_data.pkl')

In [18]:
final_data = pd.read_pickle(output_path + 'jicha_jin_vol_sel_sym_all_post_final_data.pkl')
final_data

,Date,Ticker,Open,Close,Low,High,Volume,Value,high_limit,pct_chg,open_chg,3_basis_momentum,5_basis_momentum,10_basis_momentum,15_basis_momentum,20_basis_momentum,25_basis_momentum,30_basis_momentum,50_basis_momentum,80_basis_momentum,100_basis_momentum,120_basis_momentum,150_basis_momentum,is_trade,open_lim_up,next_pct_chg,future_Ndays_chg,tom_istrade,next_day_open_lim_up,tom_open_chg
1,2015-01-06,A9999.XDCE,3948.89,3951.52,3936.60,3991.01,176706.0,7.977943e+09,4093.68,0.001335,[0.0006660099420343624],0.012956,0.002114,-0.018588,-0.016105,-0.012149,-0.007216,-0.002859,0.017658,0.055616,0.061588,0.060184,0.068445,1,False,-0.010424,"[-0.00199436025214772, -0.010424200828867858, ...",1.0,False,[-0.00199436025214772]
2,2015-01-07,A9999.XDCE,3961.17,3953.27,3931.33,3973.46,102236.0,4.604084e+09,4119.13,0.000443,[-0.00199436025214772],0.004749,0.010654,-0.020318,-0.022891,-0.017504,-0.020219,-0.004328,0.010618,0.049085,0.053990,0.055300,0.064527,1,False,0.001873,"[-0.0015514217997503188, -0.010424200828867858...",1.0,False,[-0.0015514217997503188]
3,2015-01-08,A9999.XDCE,3957.66,3951.52,3940.11,3969.95,93610.0,4.220491e+09,4109.47,-0.000443,[-0.0015514217997503188],0.005062,0.015316,-0.011247,-0.020676,-0.015293,-0.014160,-0.007133,0.024076,0.059060,0.061989,0.062466,0.074866,1,False,0.000000,"[0.00021992411354165853, -0.010424200828867858...",1.0,False,[0.00021992411354165853]
4,2015-01-09,A9999.XDCE,3955.91,3956.78,3936.60,3965.56,106146.0,4.779245e+09,4113.86,0.001331,[0.00021992411354165853],-0.001120,0.008787,-0.001431,-0.026470,-0.015660,-0.026935,-0.018675,0.021654,0.053825,0.058060,0.059666,0.070893,1,False,-0.023598,"[-0.00022230418285529296, -0.01042420082886785...",1.0,False,[-0.00022230418285529296]
5,2015-01-12,A9999.XDCE,3958.54,3957.66,3946.25,3982.23,134590.0,6.084913e+09,4108.60,0.000222,[-0.00022230418285529296],0.003797,0.001374,0.001679,-0.027297,-0.015670,-0.024185,-0.010388,0.015367,0.053901,0.056841,0.059554,0.070992,1,False,-0.001674,"[-0.003997711253342029, -0.010424200828867858,...",1.0,False,[-0.003997711253342029]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,2022-09-27,ZN9999.XSGE,24413.30,24577.40,24341.50,24808.10,162221.0,1.943660e+10,27632.90,0.006087,[0.006721745933569112],0.001315,0.000701,-0.000101,0.003647,0.004376,0.004698,0.001355,0.006309,0.009830,0.015849,0.014354,0.015923,1,False,NaN,"[-0.03264744892371707, 0.02257079842679488, -0...",1.0,False,[-0.03264744892371707]
1884,2022-09-28,ZN9999.XSGE,24654.30,23849.40,23849.40,24654.30,236290.0,2.789449e+10,27514.90,-0.029621,[-0.03264744892371707],0.003591,0.007104,0.003787,0.007001,0.007908,0.008320,0.007503,0.009275,0.017489,0.017881,0.021758,0.006196,1,False,NaN,"[0.015587130518793568, -0.004834404228361122, ...",1.0,False,[0.015587130518793568]
1885,2022-09-29,ZN9999.XSGE,24013.40,24387.70,24013.40,24459.40,182021.0,2.153816e+10,27109.90,0.022571,[0.015587130518793568],0.001473,0.004254,0.003571,0.005454,0.008235,0.007108,0.007667,0.009608,0.016469,0.018652,0.026621,0.019410,1,False,NaN,"[-0.017023017322732592, 0.026823459608237288, ...",1.0,False,[-0.017023017322732592]
1886,2022-09-30,ZN9999.XSGE,24690.10,24269.80,24259.50,24792.70,172268.0,2.057620e+10,27171.50,-0.004834,[-0.017023017322732592],0.012926,0.012587,0.010224,0.015396,0.017874,0.016675,0.017468,0.019563,0.027012,0.033058,0.018370,0.035908,1,False,NaN,"[0.02466181489247976, nan, nan, nan, nan, nan,...",1.0,False,[0.02466181489247976]


In [19]:
# final_data2 = pd.DataFrame()
# for code,df0 in final_data.groupby("Ticker"):
#     temp_df = df0.sort_values("Date")
#     for n in [10,20,30]:
#         temp_df[f"ma{n}"] = temp_df['Close'].rolling(n,min_periods=1).mean()
#     final_data2 = final_data2.append(temp_df)
# final_data2

In [20]:
# final_data2.to_pickle(output_path + 'jin_vol_ma_fiter_rolling_all_post_final_data.pkl')

In [176]:
fac = f'100_basis_momentum'


begin_day = '2015-01-12'

# ['2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09',
# '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16',
# '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23',
# '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30',
# ]

period_t = 5
fangxiang = 'long'

final_data1 = final_data.copy()

final_data1['my_factor'] = final_data1[fac]

# params_dict = {
#     'begin_date':begin_day,
#     'period_type':period_t,
#     'num':5,
#     'rank':0,
#     'fangxiang':fangxiang,
#     'c_rate': 1.5 / 10000,  # 手续费
#     't_rate':  0 / 1000,  # 印花税
# }

if fangxiang == "short":
    params_dict = {
        'begin_date': begin_day,
        'period_type': period_t,
        'num': 5,
        'rank': 5,
        'fangxiang': fangxiang,
        'c_rate': 1.5 / 10000,  # 手续费
        't_rate': 0 / 1000,  # 印花税
    }
else:
    params_dict = {
        'begin_date': begin_day,
        'period_type': period_t,
        'num': 5,
        'rank': 0,
        'fangxiang': fangxiang,
        'c_rate': 1.5 / 10000,  # 手续费
        't_rate': 0 / 1000,  # 印花税
    }

final_data1 = final_data1[final_data1['Value']>=100000000]

# final_data1 = final_data1[abs(final_data1['tvalues'])>=1]
    
final_data1.reset_index(drop=True,inplace=True)
# final_data1

In [177]:
select_stock, equity= get_backtest_equity_and_select_stock_data(final_data1, is_trade_date, index_alldata, params_dict)
equity

,Date,index_pct_chg,hold_Ticker,pct_chg,equity_curve,benchmark
0,2015-01-13,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.003193,0.996807,1.0
1,2015-01-14,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,0.004544,1.001337,1.0
2,2015-01-15,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.009687,0.991637,1.0
3,2015-01-16,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.000113,0.991525,1.0
4,2015-01-19,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.014691,0.976958,1.0
...,...,...,...,...,...,...
1877,2022-09-27,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.016837,2.863055,1.0
1878,2022-09-28,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.006494,2.881648,1.0
1879,2022-09-29,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.014331,2.922945,1.0
1880,2022-09-30,0.0,M9999.XDCE OI9999.XZCE JM9999.XDCE TA9999.XZCE...,-0.002733,2.914956,1.0


In [178]:
rtn, year_return, month_return = strategy_evaluate(equity, select_stock)
rtn

,0
累积净值,2.98
年化收益,15.12%
最大回撤,-40.45%
最大回撤开始时间,2021-10-19 00:00:00
最大回撤结束时间,2021-12-10 00:00:00
年化收益/回撤比,0.37
盈利周期数,203.0
亏损周期数,173.0
胜率,53.85%
每周期平均收益,0.34%


In [179]:
line = draw_captial_curve(equity)
line.render_notebook()

In [180]:
select_stock[select_stock['ticker_next_chg'] == select_stock['ticker_next_chg'].min()]

,index,Date,buy_Ticker,Ticker_number,ticker_next_everday_chg,ticker_next_chg,price
330,330,2021-10-25,ZC9999.XZCE JM9999.XDCE J9999.XDCE SF9999.XZCE...,5,"[-0.012089677293897871, -0.0828386269084953, -...",-0.202342,2.303101


In [181]:
# 保存最终的曲线用于对冲
equity.to_pickle(output_path + f'{fangxiang}_rolling_equity.pkl')

In [182]:
good_equity = pd.read_pickle(output_path + f'long_rolling_equity.pkl')
bad_equity = pd.read_pickle(output_path + f'short_rolling_equity.pkl')
bad_equity

,Date,index_pct_chg,hold_Ticker,pct_chg,equity_curve,benchmark
0,2015-01-13,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,0.000217,1.000217,1.0
1,2015-01-14,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,0.008380,1.008599,1.0
2,2015-01-15,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.002960,1.005614,1.0
3,2015-01-16,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.001664,1.003940,1.0
4,2015-01-19,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.004964,0.998957,1.0
...,...,...,...,...,...,...
1877,2022-09-27,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,0.004062,1.421048,1.0
1878,2022-09-28,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,-0.004963,1.413996,1.0
1879,2022-09-29,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,0.006158,1.422704,1.0
1880,2022-09-30,0.0,SF9999.XZCE P9999.XDCE SC9999.XINE EG9999.XDCE...,0.002238,1.425888,1.0


In [183]:
good_equity.reset_index(inplace=True)
bad_equity.reset_index(inplace=True)
hedge_equity = pd.merge(good_equity, bad_equity,on=['Date'])
hedge_equity

,index_x,Date,index_pct_chg_x,hold_Ticker_x,pct_chg_x,equity_curve_x,benchmark_x,index_y,index_pct_chg_y,hold_Ticker_y,pct_chg_y,equity_curve_y,benchmark_y
0,0,2015-01-13,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.003193,0.996807,1.0,0,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,0.000217,1.000217,1.0
1,1,2015-01-14,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,0.004544,1.001337,1.0,1,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,0.008380,1.008599,1.0
2,2,2015-01-15,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.009687,0.991637,1.0,2,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.002960,1.005614,1.0
3,3,2015-01-16,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.000113,0.991525,1.0,3,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.001664,1.003940,1.0
4,4,2015-01-19,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.014691,0.976958,1.0,4,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.004964,0.998957,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,1877,2022-09-27,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.016837,2.863055,1.0,1877,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,0.004062,1.421048,1.0
1878,1878,2022-09-28,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.006494,2.881648,1.0,1878,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,-0.004963,1.413996,1.0
1879,1879,2022-09-29,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.014331,2.922945,1.0,1879,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,0.006158,1.422704,1.0
1880,1880,2022-09-30,0.0,M9999.XDCE OI9999.XZCE JM9999.XDCE TA9999.XZCE...,-0.002733,2.914956,1.0,1880,0.0,SF9999.XZCE P9999.XDCE SC9999.XINE EG9999.XDCE...,0.002238,1.425888,1.0


In [184]:
hedge_equity['pct_chg'] = (hedge_equity['pct_chg_x']-hedge_equity['pct_chg_y'])/2
# hedge_equity['pct_chg'] = (hedge_equity['pct_chg_x']-hedge_equity['pct_chg_y'])
hedge_equity['equity_curve'] = (hedge_equity['pct_chg'] + 1).cumprod()
hedge_equity['benchmark'] = (hedge_equity['index_pct_chg_x'] + 1).cumprod()
hedge_equity

,index_x,Date,index_pct_chg_x,hold_Ticker_x,pct_chg_x,equity_curve_x,benchmark_x,index_y,index_pct_chg_y,hold_Ticker_y,pct_chg_y,equity_curve_y,benchmark_y,pct_chg,equity_curve,benchmark
0,0,2015-01-13,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.003193,0.996807,1.0,0,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,0.000217,1.000217,1.0,-0.001705,0.998295,1.0
1,1,2015-01-14,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,0.004544,1.001337,1.0,1,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,0.008380,1.008599,1.0,-0.001918,0.996380,1.0
2,2,2015-01-15,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.009687,0.991637,1.0,2,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.002960,1.005614,1.0,-0.003364,0.993029,1.0
3,3,2015-01-16,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.000113,0.991525,1.0,3,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.001664,1.003940,1.0,0.000776,0.993799,1.0
4,4,2015-01-19,0.0,J9999.XDCE JM9999.XDCE RU9999.XSGE HC9999.XSGE...,-0.014691,0.976958,1.0,4,0.0,CF9999.XZCE RB9999.XSGE JD9999.XDCE AU9999.XSG...,-0.004964,0.998957,1.0,-0.004864,0.988965,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,1877,2022-09-27,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.016837,2.863055,1.0,1877,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,0.004062,1.421048,1.0,0.006387,1.430420,1.0
1878,1878,2022-09-28,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.006494,2.881648,1.0,1878,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,-0.004963,1.413996,1.0,0.005728,1.438614,1.0
1879,1879,2022-09-29,0.0,M9999.XDCE OI9999.XZCE BU9999.XSGE SA9999.XZCE...,0.014331,2.922945,1.0,1879,0.0,UR9999.XZCE EG9999.XDCE C9999.XDCE PB9999.XSGE...,0.006158,1.422704,1.0,0.004086,1.444492,1.0
1880,1880,2022-09-30,0.0,M9999.XDCE OI9999.XZCE JM9999.XDCE TA9999.XZCE...,-0.002733,2.914956,1.0,1880,0.0,SF9999.XZCE P9999.XDCE SC9999.XINE EG9999.XDCE...,0.002238,1.425888,1.0,-0.002485,1.440902,1.0


In [185]:
line = draw_captial_curve(hedge_equity)
line.render_notebook()

In [186]:
rtn, year_return, month_return = strategy_evaluate(hedge_equity, select_stock)
rtn

,0
累积净值,1.42
年化收益,4.65%
最大回撤,-20.78%
最大回撤开始时间,2021-10-18 00:00:00
最大回撤结束时间,2022-01-14 00:00:00
年化收益/回撤比,0.22
盈利周期数,203.0
亏损周期数,173.0
胜率,53.85%
每周期平均收益,0.34%


In [187]:
hedge_equity.to_csv(chicang_path + "jicha_jinvol" +fac+"_"+begin_day+"_"+str(period_t)+"_"+'hedge_equity.csv')
hedge_equity = hedge_equity[['Date','pct_chg','equity_curve','benchmark']]
hedge_equity.to_pickle(output_path + "jicha_jinvol" +fac+"_"+begin_day+"_"+str(period_t)+"_"+'hedge_equity.pkl')
hedge_equity

,Date,pct_chg,equity_curve,benchmark
0,2015-01-13,-0.001705,0.998295,1.0
1,2015-01-14,-0.001918,0.996380,1.0
2,2015-01-15,-0.003364,0.993029,1.0
3,2015-01-16,0.000776,0.993799,1.0
4,2015-01-19,-0.004864,0.988965,1.0
...,...,...,...,...
1877,2022-09-27,0.006387,1.430420,1.0
1878,2022-09-28,0.005728,1.438614,1.0
1879,2022-09-29,0.004086,1.444492,1.0
1880,2022-09-30,-0.002485,1.440902,1.0


In [198]:
lujing = 2
date1 = '2015-01-05'
date2 = '2015-01-06'
date3 = '2015-01-07'
date4 = '2015-01-08'
date5 = '2015-01-09'

if lujing==2:
    # 2个路径
    hedge_equity1 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date1+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity2 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date2+"_"+str(period_t)+"_"+'hedge_equity.pkl')

    all_hedge_equity = pd.merge(hedge_equity1, hedge_equity2,on=['Date'],suffixes=("_1","_2"))

    all_hedge_equity['pct_chg'] = (all_hedge_equity['pct_chg_1']+all_hedge_equity['pct_chg_2'])/2
    all_hedge_equity['equity_curve'] = (all_hedge_equity['pct_chg'] + 1).cumprod()
    all_hedge_equity['benchmark'] = all_hedge_equity['benchmark_1']
    all_all_hedge_equity = all_hedge_equity[['Date','pct_chg','equity_curve','benchmark']]
    all_all_hedge_equity
elif lujing==3:
    # 3个路径
    hedge_equity1 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date1+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity2 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date2+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity3 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date3+"_"+str(period_t)+"_"+'hedge_equity.pkl')

    all_hedge_equity = pd.merge(hedge_equity1, hedge_equity2,on=['Date'],suffixes=("_1","_2"))
    all_hedge_equity = pd.merge(all_hedge_equity, hedge_equity3,on=['Date'])

    all_hedge_equity['pct_chg'] = (all_hedge_equity['pct_chg_1']+all_hedge_equity['pct_chg_2']+all_hedge_equity['pct_chg'])/3
    all_hedge_equity['equity_curve'] = (all_hedge_equity['pct_chg'] + 1).cumprod()
    all_all_hedge_equity = all_hedge_equity[['Date','pct_chg','equity_curve','benchmark']]
    
elif lujing==4:
    # 4个路径
    hedge_equity1 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date1+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity2 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date2+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity3 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date3+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity4 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date4+"_"+str(period_t)+"_"+'hedge_equity.pkl')

    all_hedge_equity = pd.merge(hedge_equity1, hedge_equity2,on=['Date'],suffixes=("_1","_2"))
    all_hedge_equity = pd.merge(all_hedge_equity, hedge_equity3,on=['Date'])
    all_hedge_equity = pd.merge(all_hedge_equity, hedge_equity4,on=['Date'],suffixes=("_3","_4"))

    all_hedge_equity['benchmark'] = all_hedge_equity['benchmark_4']
    all_hedge_equity['pct_chg'] = (all_hedge_equity['pct_chg_1']+all_hedge_equity['pct_chg_2']+all_hedge_equity['pct_chg_3']+all_hedge_equity['pct_chg_4'])/4
    all_hedge_equity['equity_curve'] = (all_hedge_equity['pct_chg'] + 1).cumprod()
    all_all_hedge_equity = all_hedge_equity[['Date','pct_chg','equity_curve','benchmark']]
    
elif lujing==5:
    # 5个路径
    hedge_equity1 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date1+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity2 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date2+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity3 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date3+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity4 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date4+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    hedge_equity5 = pd.read_pickle(output_path+ "jicha_jinvol"+fac+"_"+date5+"_"+str(period_t)+"_"+'hedge_equity.pkl')
    all_hedge_equity = pd.merge(hedge_equity1, hedge_equity2,on=['Date'],suffixes=("_1","_2"))

    all_hedge_equity = pd.merge(all_hedge_equity, hedge_equity3,on=['Date'])

    all_hedge_equity = pd.merge(all_hedge_equity, hedge_equity4,on=['Date'],suffixes=("_3","_4"))
    all_hedge_equity = pd.merge(all_hedge_equity, hedge_equity5,on=['Date'])

    all_hedge_equity['pct_chg'] = (all_hedge_equity['pct_chg_1']+all_hedge_equity['pct_chg_2']+all_hedge_equity['pct_chg_3']+all_hedge_equity['pct_chg_4']+all_hedge_equity['pct_chg'])/5
    all_hedge_equity['equity_curve'] = (all_hedge_equity['pct_chg'] + 1).cumprod()
    all_all_hedge_equity = all_hedge_equity[['Date','pct_chg','equity_curve','benchmark']]
all_all_hedge_equity

,Date,pct_chg,equity_curve,benchmark
0,2015-01-07,0.000956,1.000956,1.0
1,2015-01-08,0.002882,1.003841,1.0
2,2015-01-09,0.000885,1.004730,1.0
3,2015-01-12,0.001321,1.006057,1.0
4,2015-01-13,-0.005359,1.000665,1.0
...,...,...,...,...
1881,2022-09-27,0.004863,1.559887,1.0
1882,2022-09-28,0.001341,1.561979,1.0
1883,2022-09-29,0.000920,1.563416,1.0
1884,2022-09-30,-0.000559,1.562542,1.0


In [199]:
line = draw_captial_curve(all_all_hedge_equity)
line.render_notebook()

In [200]:
rtn, year_return, month_return = strategy_evaluate(all_all_hedge_equity, select_stock)
rtn

,0
累积净值,1.54
年化收益,5.76%
最大回撤,-19.16%
最大回撤开始时间,2021-10-14 00:00:00
最大回撤结束时间,2022-01-13 00:00:00
年化收益/回撤比,0.3
盈利周期数,203.0
亏损周期数,173.0
胜率,53.85%
每周期平均收益,0.34%
